# Tutorial práctico de Apache Kafka (KRaft)

## Introducción y preparación del entorno

En este cuaderno se utilizará un clúster de Apache Kafka desplegado mediante contenedores Docker,
basado en la arquitectura **KRaft (Kafka Raft Metadata Mode)**, sin dependencia de ZooKeeper.

Este apartado inicial tiene como finalidad **verificar que el entorno está correctamente configurado**
y que podemos comunicarnos con el clúster Kafka.


## 1. Preparación del entorno de trabajo

El cuaderno utiliza Python para interactuar con Kafka mediante la librería
**confluent-kafka**, un wrapper de alto rendimiento basado en *librdkafka*.

Antes de comenzar, asumimos que:
- El clúster Kafka está en ejecución.
- Jupyter tiene acceso a la red donde se encuentran los brokers.
- La variable de entorno con los brokers está correctamente definida.

En el siguiente bloque se importarán las librerías necesarias
y se comprobará la conectividad básica con el clúster.


In [ ]:
#En el caso que la librería no exista, hay que instalarla
%pip install --upgrade confluent-kafka

from confluent_kafka.admin import AdminClient
from confluent_kafka import KafkaException
import os



### Configuración de conexión al clúster

Los brokers de Kafka se proporcionan mediante una variable de entorno,
lo que permite desacoplar el código del entorno físico donde se ejecuta.

Esta aproximación es habitual en entornos profesionales
y facilita el despliegue del mismo código en distintos escenarios
(desarrollo, pruebas o producción).


In [ ]:
# Obtener la lista de brokers desde la variable de entorno
brokers = os.getenv("KAFKA_BROKERS")

if not brokers:
    raise RuntimeError("No se ha definido la variable de entorno KAFKA_BROKERS")

print(f"Brokers configurados: {brokers}")


### Comprobación de conectividad con el clúster

En este primer contacto con Kafka no se producirán ni consumirán mensajes.
Únicamente se consultará la **metadata del clúster** para verificar que:

- El clúster está accesible.
- Los brokers responden correctamente.
- Podemos obtener información básica de la infraestructura.

Este tipo de comprobación es habitual como primer paso
en cualquier aplicación que vaya a interactuar con Kafka.


In [ ]:
from confluent_kafka.admin import AdminClient
import os

# Crear cliente administrativo
admin_client = AdminClient({'bootstrap.servers': brokers})

# Obtener metadatos completos del clúster
cluster_metadata = admin_client.list_topics(timeout=10)

print("=== Brokers del clúster ===")
for broker_id, broker in cluster_metadata.brokers.items():
    print(f"Broker ID: {broker_id}, Host: {broker.host}, Port: {broker.port}")

# Identificar el controller activo
controller_id = cluster_metadata.controller_id
controller = cluster_metadata.brokers[controller_id]

print("\n=== Controller activo ===")
print(f"Broker ID: {controller_id}, Host: {controller.host}, Port: {controller.port}")


### Observación en Kafdrop

Para complementar las acciones realizadas desde este cuaderno, utilizaremos **Kafdrop** como herramienta de exploración visual del clúster Kafka.

Kafdrop permite inspeccionar de forma gráfica y en tiempo real distintos elementos del clúster, como:
- Brokers disponibles
- Topics existentes
- Número de particiones y factor de replicación
- Líderes de partición
- Mensajes almacenados

Durante el desarrollo de este tutorial, tras ejecutar los distintos fragmentos de código en Python, se recomienda acceder a la interfaz web de Kafdrop para comprobar y analizar los efectos producidos sobre el clúster.

La interfaz está disponible en la siguiente URL:

👉 **http://localhost:9000/**

Esta observación cruzada entre código y visualización facilitará la comprensión de los conceptos teóricos que se van introduciendo a lo largo de la sesión.


## 2. Topics como entidad lógica distribuida

En Apache Kafka, un *topic* es una entidad lógica que representa un flujo de datos. A diferencia de otros sistemas de mensajería, un topic **no es un recurso físico localizado en un único nodo**, sino una estructura distribuida a lo largo del clúster.

Cada topic se divide en **particiones**, y cada partición se replica en uno o varios brokers. Esta arquitectura permite:

- Escalar el procesamiento de datos de forma horizontal.
- Garantizar tolerancia a fallos mediante replicación.
- Distribuir la carga de lectura y escritura entre distintos brokers.

Desde el punto de vista del cliente (productores y consumidores), el topic se percibe como una única entidad lógica, aunque internamente esté repartido entre múltiples nodos.


### Particiones y réplicas

Una **partición** es la unidad mínima de paralelismo en Kafka. Cada partición tiene:

- Un **líder**, que gestiona las operaciones de lectura y escritura.
- Una o varias **réplicas seguidoras**, que mantienen una copia sincronizada de los datos.

Si el broker que aloja al líder de una partición deja de estar disponible, Kafka puede **elegir automáticamente un nuevo líder** entre las réplicas disponibles, garantizando la continuidad del servicio.

Este comportamiento será observable más adelante tanto desde el código como desde la interfaz web de Kafdrop.


In [ ]:
from confluent_kafka.admin import AdminClient, NewTopic
import os

# Configuración de conexión
brokers = os.environ.get(
    "KAFKA_BROKERS",
    "kafka1:9092,kafka2:9094,kafka3:9096,kafka4:9098"
)

admin_client = AdminClient({'bootstrap.servers': brokers})

# Definición del topic
topic_name = "demo-topic-distribuido"

new_topic = NewTopic(
    topic=topic_name,
    num_partitions=3,
    replication_factor=2
)

# Creación del topic
fs = admin_client.create_topics([new_topic])

for topic, future in fs.items():
    try:
        future.result()
        print(f"Topic '{topic}' creado correctamente.")
    except Exception as e:
        print(f"Error al crear el topic '{topic}': {e}")


El topic creado tiene las siguientes características:

- **3 particiones**, lo que permite paralelizar la producción y el consumo.
- **Factor de replicación 2**, lo que implica que cada partición estará presente en dos brokers distintos.

A partir de este momento, el topic existe físicamente repartido entre varios brokers del clúster. Cada partición tendrá un broker líder asignado automáticamente por Kafka.


In [ ]:
# Obtener metadatos del topic creado
metadata = admin_client.list_topics(timeout=10)

topic_metadata = metadata.topics[topic_name]

print(f"=== Detalle del topic '{topic_name}' ===")

for partition_id, partition in topic_metadata.partitions.items():
    print(
        f"Partición {partition_id} | "
        f"Líder: {partition.leader} | "
        f"Réplicas: {partition.replicas} | "
        f"ISR: {partition.isrs}"
    )


En la salida anterior se puede observar:

- El **broker líder** de cada partición.
- La lista de **réplicas** asignadas.
- El conjunto ISR (*In-Sync Replicas*), que indica qué réplicas están correctamente sincronizadas.

Estos valores pueden cambiar dinámicamente si un broker deja de estar disponible o se reincorpora al clúster.


### Observación en Kafdrop

Accede a la interfaz web de Kafdrop:

http://localhost:9000/

Dentro del topic `demo-topic-distribuido`, revisa:

- El número de particiones.
- Qué broker actúa como líder de cada partición.
- Cómo se distribuyen las réplicas entre los distintos brokers.

Contrasta esta información con la obtenida mediante el código Python para reforzar la comprensión del carácter distribuido de los topics en Kafka.


## 3. Productores: introducción al envío de mensajes en Kafka

Hasta ahora hemos analizado Kafka desde un punto de vista estructural: el clúster, los brokers y los topics.
En este apartado damos el siguiente paso natural: **empezar a generar datos reales**.

Un **productor** es el componente encargado de enviar mensajes a Kafka.  
Desde el punto de vista del cliente, Kafka actúa como un sistema distribuido capaz de:

- recibir mensajes de forma concurrente,
- almacenarlos de manera tolerante a fallos,
- y distribuirlos entre los brokers del clúster.

En este primer contacto trabajaremos con productores de forma **intencionalmente sencilla**, centrándonos únicamente en:
- enviar mensajes de texto,
- verificar su llegada al clúster,
- observar el resultdos posteriores.


### Conceptos clave del productor

Antes de pasar al código, conviene fijar algunos conceptos básicos:

- Un productor **no envía mensajes a un broker concreto**, sino a un *topic*.
- Kafka se encarga internamente de decidir:
  - a qué partición se asigna el mensaje,
  - en qué broker se almacena.
- El envío de mensajes es **asíncrono** por defecto.
- El productor recibe confirmación de que el mensaje ha sido aceptado por el clúster.

En este punto del tutorial asumimos:
- un clúster Kafka ya operativo,
- al menos un topic existente,
- acceso al clúster mediante la librería `confluent-kafka` desde Python.


In [ ]:
from confluent_kafka import Producer
import socket


### Configuración básica del productor

Para crear un productor necesitamos, como mínimo, indicar:
- la lista de brokers del clúster (bootstrap servers),
- un identificador de cliente.

No es necesario especificar particiones ni brokers concretos.
Kafka se encarga de la distribución interna de los mensajes.


In [ ]:
conf = {
    "bootstrap.servers": "kafka1:9092,kafka2:9094,kafka3:9096,kafka4:9098",
    "client.id": socket.gethostname()
}

producer = Producer(conf)


### Envío de un primer mensaje

Vamos a enviar un mensaje simple de texto a un topic existente.
En este ejemplo utilizaremos un topic previamente creado en el clúster.

Tras ejecutar el código:
- el mensaje será enviado a Kafka,
- podremos comprobar su llegada desde la interfaz web de Kafdrop.


In [ ]:
topic_name = "demo-topic"

producer.produce(
    topic=topic_name,
    value="Primer mensaje enviado desde un productor en Python"
)

producer.flush()


### Observación en Kafdrop

Accede a la interfaz web de Kafdrop para comprobar el resultado del envío:

http://localhost:9000/

Desde Kafdrop puedes:
- localizar el topic utilizado,
- visualizar los mensajes almacenados,
- comprobar en qué partición se ha escrito elter Kafka.


## 4. El mensaje en Kafka: clave, valor y distribución

Hasta este punto del tutorial hemos trabajado con el clúster, los brokers y los topics como entidades distribuidas.
El siguiente paso natural es entender **qué es realmente un mensaje en Kafka** y cómo su estructura influye directamente en el
comportamiento del sistema.

En Kafka, un mensaje no es solo un “dato que se envía”, sino una unidad que:
- se almacena en una partición concreta,
- sigue reglas de orden,
- y participa en mecanismos de escalabilidad y tolerancia a fallos.

En este apartado nos centraremos en dos elementos clave del mensaje:
- **key (clave)**
- **value (valor)**



### Estructura básica de un mensaje Kafka

Conceptualmente, un mensaje Kafka puede representarse como:

- **key**: valor opcional que Kafka utiliza para decidir la partición destino.
- **value**: el contenido del mensaje (texto, binario, JSON, CSV, etc.).
- **offset**: posición del mensaje dentro de la partición (asignado por Kafka).
- **timestamp**: marca temporal asociada al mensaje.

En este bloque nos centraremos en la relación entre **key**, **particiones** y **orden**.


### Clave y particionado

Kafka distribuye los mensajes de un topic entre sus particiones siguiendo estas reglas generales:

- **Mensajes sin clave**  
  Kafka distribuye los mensajes entre las particiones disponibles de forma automática.
  El objetivo es equilibrar la carga.

- **Mensajes con clave**  
  Kafka calcula un hash de la clave y siempre envía los mensajes con la misma clave
  a la **misma partición**.

Esto implica dos consecuencias importantes:
- El orden **solo está garantizado dentro de una partición**.
- La clave es una herramienta fundamental para controlar el orden lógico de los mensajes.


In [ ]:
from confluent_kafka import Producer
import time

brokers = "kafka1:9092,kafka2:9094,kafka3:9096,kafka4:9098"
topic = "demo-topic-distribuido"

producer = Producer({
    "bootstrap.servers": brokers
})


### Envío de mensajes sin clave

Vamos a enviar varios mensajes **sin clave** al mismo topic.
Kafka decidirá automáticamente a qué partición va cada uno.

Después de ejecutar la celda, observa el resultado en Kafdrop:
- topic `demo-topic-distribuido`
- pestaña *Topics → Partitions*


In [ ]:
for i in range(10):
    producer.produce(
        topic=topic,
        value=f"Mensaje sin clave {i}"
    )

producer.flush()


### Observación en Kafdrop

Accede a la interfaz web de Kafdrop:

http://localhost:9000/

Observa:
- cómo los mensajes se reparten entre distintas particiones,
- que no existe una relación directa entre el contenido del mensaje y la partición.

Esto es un comportamiento normal cuando **no se define clave**.


### Envío de mensajes con clave

Ahora enviaremos mensajes **con la misma clave**.
El objetivo es comprobar que Kafka los dirige siempre a la misma partición.


In [ ]:
for i in range(10):
    producer.produce(
        topic=topic,
        key="usuario-123",
        value=f"Mensaje con clave usuario-123 #{i}"
    )

producer.flush()


### Observación del efecto de la clave

En Kafdrop, revisa nuevamente las particiones del topic.

Deberías observar que:
- todos los mensajes con la clave `usuario-123` están en la **misma partición**,
- el orden de los mensajes se mantiene dentro de esa partición.

Esto ilustra una regla fundamental de Kafka:
> El orden está garantizado por partición, no por topic.


### Múltiples claves, múltiples flujos

Por último, enviaremos mensajes con **claves distintas** para simular varios flujos de datos independientes.


In [ ]:
claves = ["cliente-A", "cliente-B", "cliente-C"]

for clave in claves:
    for i in range(5):
        producer.produce(
            topic=topic,
            key=clave,
            value=f"Evento {i} para {clave}"
        )

producer.flush()


### Qué debe quedar claro tras este apartado

- La clave no es obligatoria, pero **sí estratégica**.
- Kafka no garantiza orden global, solo **orden por partición**.
- Elegir bien la clave impacta directamente en:
  - escalabilidad,
  - orden,
  - diseño de consumidores.

Estos conceptos serán fundamentales cuando pasemos a:
- consumidores,
- procesamiento paralelo,
- y diseño de aplicaciones reales con Kafka.


## 5. El consumidor en Kafka

Hasta ahora hemos visto cómo se producen mensajes y cómo estos se distribuyen en el clúster.
El siguiente elemento clave es el **consumidor**, responsable de leer los mensajes almacenados en los topics.

En Kafka, consumir no significa “recibir un mensaje y borrarlo”, sino:
- leer mensajes de una partición,
- mantener la posición de lectura (offset),
- y coordinarse con otros consumidores cuando trabajan en grupo.

Este apartado se centrará en:
- cómo se leen los mensajes,
- qué es un offset,
- y cómo Kafka gestiona el consumo distribuido.


### Conceptos básicos del consumo

Al consumir mensajes en Kafka debemos tener claros los siguientes conceptos:

- **Offset**  
  Es la posición de lectura dentro de una partición.
  Kafka no borra los mensajes cuando se consumen; el consumidor simplemente avanza su offset.

- **Lectura secuencial**  
  Los mensajes se leen en orden dentro de cada partición.

- **Estado del consumidor**  
  Kafka guarda el offset asociado a un consumidor (o grupo) para saber por dónde continuar.


In [ ]:
from confluent_kafka import Consumer

### Configuración básica de un consumidor

Vamos a crear un consumidor sencillo que:
- se conecte al clúster,
- lea mensajes de un topic,
- y muestre su contenido por pantalla.

En este primer ejemplo no usaremos grupos complejos.


In [ ]:
brokers = "kafka1:9092,kafka2:9094,kafka3:9096,kafka4:9098"
topic = "demo-topic-distribuido"

consumer_conf = {
    "bootstrap.servers": brokers,
    "group.id": "grupo-demo",
    "auto.offset.reset": "earliest"
}

consumer = Consumer(consumer_conf)
consumer.subscribe([topic])


### Lectura de mensajes

A continuación leeremos mensajes del topic.
Observa que:
- el consumidor va leyendo mensajes uno a uno,
- cada mensaje pertenece a una partición concreta,
- y tiene un offset asociado.


In [ ]:
print("Leyendo mensajes...")

for _ in range(10):
    msg = consumer.poll(1.0)

    if msg is None:
        continue

    if msg.error():
        print(f"Error: {msg.error()}")
    else:
        print(
            f"Partición: {msg.partition()} | "
            f"Offset: {msg.offset()} | "
            f"Clave: {msg.key()} | "
            f"Valor: {msg.value().decode('utf-8')}"
        )

consumer.close()


### Observación en Kafdrop

Mientras el consumidor está activo o después de ejecutar la celda, revisa en Kafdrop:

http://localhost:9000/

Observa:
- el número de mensajes en cada partición,
- cómo el offset va avanzando,
- que los mensajes no se eliminan tras ser consumidos.


### El offset y la relectura de mensajes

El valor `auto.offset.reset` controla desde dónde empieza a leer un consumidor cuando
no existe un offset previo guardado:

- **earliest**: desde el primer mensaje disponible.
- **latest**: solo mensajes nuevos a partir de ese momento.

Este comportamiento es clave para:
- pruebas,
- reprocesamiento,
- y depuración.


In [ ]:
# Ejemplo alternativo: comenzar desde los mensajes más recientes

consumer_conf_latest = {
    "bootstrap.servers": brokers,
    "group.id": "grupo-demo-latest",
    "auto.offset.reset": "latest"
}

consumer_latest = Consumer(consumer_conf_latest)
consumer_latest.subscribe([topic])

msg = consumer_latest.poll(2.0)

if msg is None:
    print("No hay mensajes nuevos")
elif msg.error():
    print(f"Error: {msg.error()}")
else:
    print(f"Mensaje recibido: {msg.value().decode('utf-8')}")

consumer_latest.close()


### Qué debe quedar claro tras este apartado

- Kafka no borra los mensajes al consumirlos.
- El consumidor controla su progreso mediante offsets.
- El orden está garantizado dentro de cada partición.
- El comportamiento del consumidor depende de su configuración.

En el siguiente apartado daremos un paso más:
veremos cómo **varios consumidores pueden trabajar juntos** mediante grupos de consumo.


## 6. Grupos de consumidores y consumo distribuido

Hasta ahora hemos trabajado con un único consumidor leyendo mensajes de un topic.
Sin embargo, uno de los pilares de Kafka es la **lectura paralela y escalable** de datos.

Esto se consigue mediante los **grupos de consumidores** (consumer groups).

Un grupo de consumidores permite:
- repartir las particiones de un topic entre varios consumidores,
- escalar horizontalmente el procesamiento,
- garantizar que cada mensaje sea procesado una sola vez por grupo.


### Conceptos clave

Antes de pasar a la práctica, conviene fijar algunos conceptos:

- **Consumer Group**  
  Conjunto de consumidores que comparten un mismo `group.id`.

- **Asignación de particiones**  
  Kafka asigna cada partición de un topic a un único consumidor dentro del grupo.

- **Paralelismo real**  
  El paralelismo máximo está limitado por el número de particiones, no por el número de consumidores.

Si hay más consumidores que particiones, algunos consumidores quedarán inactivos.


### Escenario de prueba

Para este apartado se asume:
- un topic con varias particiones,
- mensajes ya existentes en el topic,
- y varios consumidores que se lanzan con el mismo `group.id`.

El objetivo es observar cómo Kafka reparte el trabajo.


In [ ]:
from confluent_kafka import Consumer
import time

### Consumidor A (miembro del grupo)

Este será el primer consumidor del grupo.


In [ ]:
brokers = "kafka1:9092,kafka2:9094,kafka3:9096,kafka4:9098"
topic = "demo-topic-distribuido"

consumer_a_conf = {
    "bootstrap.servers": brokers,
    "group.id": "grupo-distribuido",
    "auto.offset.reset": "earliest"
}

consumer_a = Consumer(consumer_a_conf)
consumer_a.subscribe([topic])

print("Consumidor A activo")


### Consumidor B (mismo grupo)

Este segundo consumidor pertenece al mismo grupo.
Kafka redistribuirá automáticamente las particiones.


In [ ]:
consumer_b_conf = {
    "bootstrap.servers": brokers,
    "group.id": "grupo-distribuido",
    "auto.offset.reset": "earliest"
}

consumer_b = Consumer(consumer_b_conf)
consumer_b.subscribe([topic])

print("Consumidor B activo")


### Lectura simultánea desde ambos consumidores

Ejecuta esta celda en cada consumidor (A y B) y observa:

- qué particiones procesa cada uno,
- cómo no se duplican los mensajes,
- y cómo el reparto depende del número de particiones.


In [ ]:
def leer_mensajes(consumer, nombre):
    print(f"Leyendo mensajes para: {nombre}")
    for _ in range(5):
        msg = consumer.poll(1.0)

        if msg is None:
            continue

        if msg.error():
            print(f"{nombre} - Error: {msg.error()}")
        else:
            print(
                f"{nombre} | "
                f"Partición: {msg.partition()} | "
                f"Offset: {msg.offset()} | "
                f"Valor: {msg.value().decode('utf-8')}"
            )


In [ ]:
# Lectura desde el consumidor A
leer_mensajes(consumer_a, "Consumidor A")
consumer_a.close()

# Lectura desde el consumidor B
leer_mensajes(consumer_b, "Consumidor B")
consumer_b.close()

### Rebalanceo de consumidores

Kafka gestiona automáticamente los cambios en el grupo:

- si un consumidor se detiene,
- si se añade uno nuevo,
- o si cambia el número de particiones.

Este proceso se llama **rebalanceo**.

Durante el rebalanceo:
- las particiones se reasignan,
- los consumidores pausan momentáneamente la lectura,
- y luego continúan desde el offset correcto.


### Observación en Kafdrop

Accede a:

http://localhost:9000/

Observa:
- el número de particiones del topic,
- el líder de cada partición,
- y cómo el consumo se distribuye entre consumidores.

Aunque Kafdrop no muestra directamente los grupos,
sí permite razonar sobre el reparto observado en el código.


### Qué debe quedar claro tras este apartado

- Un grupo de consumidores permite escalar el procesamiento.
- Cada partición solo puede ser consumida por un consumidor del grupo.
- El paralelismo depende del número de particiones.
- Kafka gestiona automáticamente fallos y redistribuciones.

Con esto se completa el ciclo básico:
topic → particiones → productor → mensajes → consumidor → grupo.
